In [1]:
import json
import time
import requests
from collections import defaultdict


In [2]:
# Load your central panel
with open("central_metabolism_panel.json") as f:
    panel = json.load(f)

# Map gene -> reactions
gene_to_rxns = {}
for r in panel["reactions"]:
    for g in r["genes"]:
        gid = g["gene_id"]
        gene_to_rxns.setdefault(gid, set()).add(r["rxn_id"])

# Only keep real-looking Ensembl gene IDs
ensembl_genes = sorted(g for g in gene_to_rxns if g.startswith("ENSG"))
print("Ensembl-looking genes:", len(ensembl_genes))


Ensembl-looking genes: 145


In [ ]:
# FIXED: Ensembl -> UniProt mapping
# The key fix: Ensembl uses "Uniprot_gn" not "UniProtKB" in their xrefs

gene_to_uniprot = {}
uniprot_to_genes = {}
failed_genes = []

for i, gid in enumerate(ensembl_genes):
    if i % 10 == 0:
        print(f"Processing gene {i+1}/{len(ensembl_genes)}")
    
    url = f"https://rest.ensembl.org/xrefs/id/{gid}?content-type=application/json"
    try:
        r = requests.get(url, headers={"Accept": "application/json"}, timeout=10)
        if not r.ok:
            print(f"Ensembl fail {gid}: HTTP {r.status_code}")
            failed_genes.append(gid)
            continue
        
        data = r.json()
        # Look for any database name containing 'uniprot' (case insensitive)
        accs = sorted({
            x["primary_id"]
            for x in data
            if "uniprot" in (x.get("dbname") or "").lower()
        })
        
        gene_to_uniprot[gid] = accs
        for acc in accs:
            uniprot_to_genes.setdefault(acc, set()).add(gid)
            
    except Exception as e:
        print(f"Error processing {gid}: {e}")
        failed_genes.append(gid)
    
    time.sleep(0.1)  # be polite to the API

print(f"\nResults:")
print(f"  Genes with UniProt mapping: {sum(1 for v in gene_to_uniprot.values() if v)}/{len(ensembl_genes)}")
print(f"  Unique UniProt accessions: {len(uniprot_to_genes)}")
print(f"  Failed genes: {len(failed_genes)}")


Processing gene 1/145
Processing gene 11/145
Processing gene 21/145
Processing gene 31/145
Processing gene 41/145
Processing gene 51/145
Processing gene 61/145
Processing gene 71/145
Processing gene 81/145
Processing gene 91/145
Processing gene 101/145
Processing gene 111/145
Processing gene 121/145
Processing gene 131/145
Processing gene 141/145

Results:
  Genes with UniProt mapping: 145/145
  Unique UniProt accessions: 756
  Failed genes: 0


In [4]:
uniprot_records = {}
failed_uniprot = []

for i, (acc, gene_set) in enumerate(uniprot_to_genes.items()):
    if i % 10 == 0:
        print(f"Processing UniProt {i+1}/{len(uniprot_to_genes)}")
    
    url = f"https://rest.uniprot.org/uniprotkb/{acc}.json"
    try:
        r = requests.get(url, headers={"Accept": "application/json"}, timeout=10)
        if not r.ok:
            print(f"UniProt fail {acc}: HTTP {r.status_code}")
            failed_uniprot.append(acc)
            continue
            
        u = r.json()
        seq_obj = u.get("sequence") or {}
        seq = seq_obj.get("value")
        length = seq_obj.get("length")
        
        # Get PDB IDs
        pdb_ids = [
            ref["id"]
            for ref in u.get("uniProtKBCrossReferences", [])  # Note: field name might vary
            if ref.get("database") == "PDB"
        ]
        
        # Also check the older field name
        if not pdb_ids:
            pdb_ids = [
                ref["id"]
                for ref in u.get("dbReferences", [])
                if ref.get("type") == "PDB"
            ]
        
        uniprot_records[acc] = {
            "uniprot_acc": acc,
            "sequence": seq,
            "length": length,
            "pdb_ids": sorted(set(pdb_ids)),
            "ensembl_genes": sorted(gene_set),
        }
        
    except Exception as e:
        print(f"Error processing {acc}: {e}")
        failed_uniprot.append(acc)
    

print(f"\nUniProt results:")
print(f"  Entries with data: {len(uniprot_records)}/{len(uniprot_to_genes)}")
print(f"  Failed accessions: {len(failed_uniprot)}")


Processing UniProt 1/756
Processing UniProt 11/756
Processing UniProt 21/756
Processing UniProt 31/756
Processing UniProt 41/756
Processing UniProt 51/756
Processing UniProt 61/756
Processing UniProt 71/756
Processing UniProt 81/756
Processing UniProt 91/756
Processing UniProt 101/756
Processing UniProt 111/756
Processing UniProt 121/756
Processing UniProt 131/756
Processing UniProt 141/756
Processing UniProt 151/756
Processing UniProt 161/756
Processing UniProt 171/756
Processing UniProt 181/756
Processing UniProt 191/756
Processing UniProt 201/756
Processing UniProt 211/756
Processing UniProt 221/756
Processing UniProt 231/756
Processing UniProt 241/756
Processing UniProt 251/756
Processing UniProt 261/756
Processing UniProt 271/756
Processing UniProt 281/756
Processing UniProt 291/756
Processing UniProt 301/756
Processing UniProt 311/756
Processing UniProt 321/756
Processing UniProt 331/756
Processing UniProt 341/756
Processing UniProt 351/756
Processing UniProt 361/756
Processing U

In [5]:
# Build combined records (add reactions per protein)
protein_records = []

for acc, rec in uniprot_records.items():
    rxns = sorted({
        rxn_id
        for gid in rec["ensembl_genes"]
        for rxn_id in gene_to_rxns.get(gid, [])
    })
    
    protein_records.append({
        "uniprot_acc": acc,
        "ensembl_genes": rec["ensembl_genes"],
        "reactions": rxns,
        "sequence": rec["sequence"],
        "length": rec["length"],
        "pdb_ids": rec["pdb_ids"],
    })

# Save to JSON
out_json = "central_metabolism_proteins_uniprot_pdb.json"
with open(out_json, "w") as f:
    json.dump({"proteins": protein_records}, f, indent=2)

print(f"\nWrote {out_json} with {len(protein_records)} proteins")
print(f"Proteins with PDB structures: {sum(1 for p in protein_records if p['pdb_ids'])}")



Wrote central_metabolism_proteins_uniprot_pdb.json with 756 proteins
Proteins with PDB structures: 118


In [6]:
# Load the protein data and see what PDB structures are available
with open("central_metabolism_proteins_uniprot_pdb.json") as f:
    protein_data = json.load(f)

# Collect all unique PDB IDs
all_pdb_ids = set()
for protein in protein_data["proteins"]:
    all_pdb_ids.update(protein["pdb_ids"])

print(f"Total unique PDB structures: {len(all_pdb_ids)}")
print(f"First 10 PDB IDs: {sorted(all_pdb_ids)[:10]}")

# Show which proteins have the most structures
proteins_with_pdbs = [p for p in protein_data["proteins"] if p["pdb_ids"]]
proteins_with_pdbs.sort(key=lambda x: len(x["pdb_ids"]), reverse=True)

print("\nProteins with most PDB structures:")
for p in proteins_with_pdbs[:5]:
    print(f"  {p['uniprot_acc']}: {len(p['pdb_ids'])} structures - {p['pdb_ids'][:3]}...")

Total unique PDB structures: 568
First 10 PDB IDs: ['1ALD', '1AOS', '1CZA', '1DGK', '1F05', '1FTA', '1FYC', '1HKB', '1HKC', '1HTI']

Proteins with most PDB structures:
  P14618: 56 structures - ['1T5A', '1ZJH', '3BJF']...
  P09467: 50 structures - ['1FTA', '2FHY', '2FIE']...
  O14561: 36 structures - ['2DNW', '5OOL', '5OOM']...
  P35557: 35 structures - ['1V4S', '1V4T', '3A0I']...
  P49327: 33 structures - ['1XKT', '2CG5', '2JFD']...


In [8]:
import os
import time
import requests

# Create directory for PDB files
pdb_dir = "pdb_structures"
os.makedirs(pdb_dir, exist_ok=True)

# Download PDB files
failed_downloads = []
downloaded = []

for i, pdb_id in enumerate(sorted(all_pdb_ids)):
    if i % 10 == 0:
        print(f"Downloading PDB {i+1}/{len(all_pdb_ids)}")
    
    # RCSB PDB download URL
    url = f"https://files.rcsb.org/download/{pdb_id}.pdb"
    output_file = os.path.join(pdb_dir, f"{pdb_id}.pdb")
    
    # Skip if already downloaded
    if os.path.exists(output_file):
        print(f"  {pdb_id}: already exists, skipping")
        downloaded.append(pdb_id)
        continue
    
    try:
        r = requests.get(url, timeout=10)
        if r.ok:
            with open(output_file, "w") as f:
                f.write(r.text)
            downloaded.append(pdb_id)
        else:
            print(f"  {pdb_id}: HTTP {r.status_code}")
            failed_downloads.append((pdb_id, f"HTTP {r.status_code}"))
    except Exception as e:
        print(f"  {pdb_id}: Error - {e}")
        failed_downloads.append((pdb_id, str(e)))
    

print(f"\nDownload complete:")
print(f"  Successfully downloaded: {len(downloaded)}/{len(all_pdb_ids)}")
print(f"  Failed: {len(failed_downloads)}")

if failed_downloads:
    print("\nFailed downloads:")
    for pdb_id, error in failed_downloads[:10]:
        print(f"  {pdb_id}: {error}")

  1ALD: already exists, skipping
  1AOS: already exists, skipping
  1CZA: already exists, skipping
  1DGK: already exists, skipping
  1F05: already exists, skipping
  1FTA: already exists, skipping
  1FYC: already exists, skipping
  1HKB: already exists, skipping
  1HKC: already exists, skipping
  5XTH: HTTP 404
  5XTI: HTTP 404
  7A5H: HTTP 404
  7A5J: HTTP 404
  7MHE: HTTP 404
  7O9K: HTTP 404
  7O9M: HTTP 404
  7ODR: HTTP 404
  7ODS: HTTP 404
  7ODT: HTTP 404
  7OF2: HTTP 404
  7OF3: HTTP 404
  7OF5: HTTP 404
  7OF7: HTTP 404
  7OIC: HTTP 404
  7OID: HTTP 404
  7PD3: HTTP 404
  7PO4: HTTP 404
  8H9E: HTTP 404
  8H9I: HTTP 404
  8H9L: HTTP 404
  8H9P: HTTP 404
  8H9S: HTTP 404
  8H9T: HTTP 404
  8H9U: HTTP 404
  8K2B: HTTP 404
  8P2K: HTTP 404
  8P5F: HTTP 404
  8PIU: HTTP 404
  8PK0: HTTP 404
  8QSJ: HTTP 404

Download complete:
  Successfully downloaded: 537/568
  Failed: 31

Failed downloads:
  5XTH: HTTP 404
  5XTI: HTTP 404
  7A5H: HTTP 404
  7A5J: HTTP 404
  7MHE: HTTP 404
  7O

In [9]:
# Create a mapping file that links PDB structures to proteins and reactions
pdb_mapping = {}

for protein in protein_data["proteins"]:
    for pdb_id in protein["pdb_ids"]:
        if pdb_id not in pdb_mapping:
            pdb_mapping[pdb_id] = {
                "pdb_id": pdb_id,
                "proteins": [],
                "reactions": set(),
                "genes": set()
            }
        
        pdb_mapping[pdb_id]["proteins"].append(protein["uniprot_acc"])
        pdb_mapping[pdb_id]["reactions"].update(protein["reactions"])
        pdb_mapping[pdb_id]["genes"].update(protein["ensembl_genes"])

# Convert sets to lists for JSON serialization
for pdb_id in pdb_mapping:
    pdb_mapping[pdb_id]["reactions"] = sorted(pdb_mapping[pdb_id]["reactions"])
    pdb_mapping[pdb_id]["genes"] = sorted(pdb_mapping[pdb_id]["genes"])
    pdb_mapping[pdb_id]["proteins"] = sorted(set(pdb_mapping[pdb_id]["proteins"]))

# Save mapping
with open("pdb_to_metabolism_mapping.json", "w") as f:
    json.dump({"pdb_structures": list(pdb_mapping.values())}, f, indent=2)

print(f"Saved mapping for {len(pdb_mapping)} PDB structures")

# Show some examples
print("\nExample mappings:")
for pdb_data in list(pdb_mapping.values())[:3]:
    print(f"\nPDB {pdb_data['pdb_id']}:")
    print(f"  Proteins: {pdb_data['proteins']}")
    print(f"  Reactions: {pdb_data['reactions'][:3]}..." if len(pdb_data['reactions']) > 3 else f"  Reactions: {pdb_data['reactions']}")

Saved mapping for 568 PDB structures

Example mappings:

PDB 2DNW:
  Proteins: ['O14561']
  Reactions: ['CI_MitoCore']

PDB 5OOL:
  Proteins: ['O14561']
  Reactions: ['CI_MitoCore']

PDB 5OOM:
  Proteins: ['O14561']
  Reactions: ['CI_MitoCore']


In [16]:
import json
import pandas as pd

# Load the data
with open("central_metabolism_panel.json") as f:
    panel = json.load(f)

with open("central_metabolism_proteins_uniprot_pdb.json") as f:
    protein_data = json.load(f)

# Build lookup tables
gene_to_proteins = {}
protein_to_pdbs = {}

for protein in protein_data["proteins"]:
    for gene in protein["ensembl_genes"]:
        if gene not in gene_to_proteins:
            gene_to_proteins[gene] = []
        gene_to_proteins[gene].append(protein["uniprot_acc"])
    protein_to_pdbs[protein["uniprot_acc"]] = protein["pdb_ids"]

# Build the clean table
rows = []

for rxn in panel["reactions"]:
    # Build clean reaction equation
    reactants = []
    products = []
    
    for met in rxn["stoichiometry"]:
        # Clean metabolite name
        name = met['met_name']
        coeff = abs(met['coeff'])
        
        if coeff == 1.0:
            met_str = name
        elif coeff == int(coeff):
            met_str = f"{int(coeff)} {name}"
        else:
            met_str = f"{coeff} {name}"
        
        if met['coeff'] < 0:
            reactants.append(met_str)
        else:
            products.append(met_str)
    
    equation = " + ".join(reactants) + " --> " + " + ".join(products)
    
    # Count genes
    gene_count = len(rxn["genes"])
    
    # Count unique PDBs for this reaction
    all_pdbs = set()
    for gene in rxn["genes"]:
        gene_id = gene["gene_id"]
        for prot in gene_to_proteins.get(gene_id, []):
            all_pdbs.update(protein_to_pdbs.get(prot, []))
    
    pdb_count = len(all_pdbs)
    
    # Add row
    rows.append({
        "Subsystem": rxn["subsystem"],
        "Reaction ID": rxn["rxn_id"],
        "Reaction": equation,
        "Gene Count": gene_count,
        "PDB Count": pdb_count
    })

# Create DataFrame and save
df = pd.DataFrame(rows)

# Sort by subsystem and reaction ID
df = df.sort_values(["Subsystem", "Reaction ID"])

# Save to CSV
df.to_csv("central_metabolism_clean.csv", index=False)

# Also display it
print("Preview of central_metabolism_clean.csv:")
print(df.to_string(index=False))

# Show summary stats
print(f"\nSummary:")
print(f"Total reactions: {len(df)}")
print(f"Reactions with PDB structures: {(df['PDB Count'] > 0).sum()}")
print(f"Total unique PDB structures: {df['PDB Count'].sum()}")
print(f"Average genes per reaction: {df['Gene Count'].mean():.1f}")
print(f"Average PDBs per reaction (when present): {df[df['PDB Count'] > 0]['PDB Count'].mean():.1f}")

Preview of central_metabolism_clean.csv:
                                   Subsystem       Reaction ID                                                                                                                                  Reaction  Gene Count  PDB Count
                          Acetate production            ACOAHi                                                                                                    H2O + Acetyl-CoA --> H + CoA + Acetate           1          3
                          Alcohol metabolism               ACS                                                                                    ATP + CoA + Acetate --> Acetyl-CoA + AMP + Diphosphate           1          0
                          Alcohol metabolism              ACSm                                                                                    ATP + CoA + Acetate --> Acetyl-CoA + AMP + Diphosphate           1          9
                        Butanoate metabolism        FACOAL40im 

In [17]:
import json
import pandas as pd

# Load all three JSON files
with open("central_metabolism_panel.json") as f:
    panel = json.load(f)

with open("central_metabolism_proteins_uniprot_pdb.json") as f:
    protein_data = json.load(f)

with open("pdb_to_metabolism_mapping.json") as f:
    pdb_mapping = json.load(f)

# Build lookup dictionaries
protein_dict = {p["uniprot_acc"]: p for p in protein_data["proteins"]}
pdb_dict = {p["pdb_id"]: p for p in pdb_mapping["pdb_structures"]}

# Create the master table - one row per gene-reaction-protein-pdb combination
rows = []

for rxn in panel["reactions"]:
    # Build reaction equation
    reactants = []
    products = []
    
    for met in rxn["stoichiometry"]:
        name = met['met_name']
        coeff = abs(met['coeff'])
        compartment = met['compartment']
        
        if coeff == 1.0:
            met_str = name
        elif coeff == int(coeff):
            met_str = f"{int(coeff)} {name}"
        else:
            met_str = f"{coeff} {name}"
        
        # Add compartment
        met_str += f" [{compartment[0]}]"  # First letter of compartment
        
        if met['coeff'] < 0:
            reactants.append(met_str)
        else:
            products.append(met_str)
    
    equation = " + ".join(reactants) + " --> " + " + ".join(products)
    
    # Get metabolite IDs for reference
    metabolite_ids = ";".join([m["met_id"] for m in rxn["stoichiometry"]])
    
    # Process each gene
    if rxn["genes"]:
        for gene in rxn["genes"]:
            gene_id = gene["gene_id"]
            
            # Find proteins for this gene
            proteins_for_gene = [p for p in protein_data["proteins"] 
                                if gene_id in p["ensembl_genes"]]
            
            if proteins_for_gene:
                for protein in proteins_for_gene:
                    # If protein has PDB structures
                    if protein["pdb_ids"]:
                        for pdb_id in protein["pdb_ids"]:
                            # Create row with full information
                            row = {
                                # Reaction info
                                "reaction_id": rxn["rxn_id"],
                                "reaction_name": rxn["rxn_name"],
                                "reaction_equation": equation,
                                "subsystem": rxn["subsystem"],
                                "metabolite_ids": metabolite_ids,
                                "num_metabolites": len(rxn["stoichiometry"]),
                                
                                # Gene info
                                "gene_id": gene_id,
                                "gene_annotation": str(gene.get("annotation", {})),
                                
                                # Protein info
                                "uniprot_acc": protein["uniprot_acc"],
                                "sequence_length": protein["length"],
                                "sequence": protein["sequence"][:50] + "..." if len(protein["sequence"]) > 50 else protein["sequence"],
                                "full_sequence": protein["sequence"],
                                
                                # PDB info
                                "pdb_id": pdb_id,
                                "has_structure": True,
                                
                                # Counts
                                "total_genes_in_reaction": len(rxn["genes"]),
                                "total_pdbs_for_protein": len(protein["pdb_ids"]),
                                "all_pdb_ids_for_protein": ";".join(protein["pdb_ids"]),
                                "all_genes_for_protein": ";".join(protein["ensembl_genes"]),
                                "all_reactions_for_protein": ";".join(protein["reactions"])
                            }
                            rows.append(row)
                    else:
                        # Protein without PDB
                        row = {
                            # Reaction info
                            "reaction_id": rxn["rxn_id"],
                            "reaction_name": rxn["rxn_name"],
                            "reaction_equation": equation,
                            "subsystem": rxn["subsystem"],
                            "metabolite_ids": metabolite_ids,
                            "num_metabolites": len(rxn["stoichiometry"]),
                            
                            # Gene info
                            "gene_id": gene_id,
                            "gene_annotation": str(gene.get("annotation", {})),
                            
                            # Protein info
                            "uniprot_acc": protein["uniprot_acc"],
                            "sequence_length": protein["length"],
                            "sequence": protein["sequence"][:50] + "..." if len(protein["sequence"]) > 50 else protein["sequence"],
                            "full_sequence": protein["sequence"],
                            
                            # PDB info
                            "pdb_id": None,
                            "has_structure": False,
                            
                            # Counts
                            "total_genes_in_reaction": len(rxn["genes"]),
                            "total_pdbs_for_protein": 0,
                            "all_pdb_ids_for_protein": "",
                            "all_genes_for_protein": ";".join(protein["ensembl_genes"]),
                            "all_reactions_for_protein": ";".join(protein["reactions"])
                        }
                        rows.append(row)
            else:
                # Gene without protein mapping
                row = {
                    # Reaction info
                    "reaction_id": rxn["rxn_id"],
                    "reaction_name": rxn["rxn_name"],
                    "reaction_equation": equation,
                    "subsystem": rxn["subsystem"],
                    "metabolite_ids": metabolite_ids,
                    "num_metabolites": len(rxn["stoichiometry"]),
                    
                    # Gene info
                    "gene_id": gene_id,
                    "gene_annotation": str(gene.get("annotation", {})),
                    
                    # Protein info
                    "uniprot_acc": None,
                    "sequence_length": None,
                    "sequence": None,
                    "full_sequence": None,
                    
                    # PDB info
                    "pdb_id": None,
                    "has_structure": False,
                    
                    # Counts
                    "total_genes_in_reaction": len(rxn["genes"]),
                    "total_pdbs_for_protein": 0,
                    "all_pdb_ids_for_protein": "",
                    "all_genes_for_protein": "",
                    "all_reactions_for_protein": ""
                }
                rows.append(row)
    else:
        # Reaction without genes
        row = {
            # Reaction info
            "reaction_id": rxn["rxn_id"],
            "reaction_name": rxn["rxn_name"],
            "reaction_equation": equation,
            "subsystem": rxn["subsystem"],
            "metabolite_ids": metabolite_ids,
            "num_metabolites": len(rxn["stoichiometry"]),
            
            # Gene info
            "gene_id": None,
            "gene_annotation": None,
            
            # Protein info
            "uniprot_acc": None,
            "sequence_length": None,
            "sequence": None,
            "full_sequence": None,
            
            # PDB info
            "pdb_id": None,
            "has_structure": False,
            
            # Counts
            "total_genes_in_reaction": 0,
            "total_pdbs_for_protein": 0,
            "all_pdb_ids_for_protein": "",
            "all_genes_for_protein": "",
            "all_reactions_for_protein": ""
        }
        rows.append(row)

# Create DataFrame
df = pd.DataFrame(rows)

# Sort by subsystem, reaction, gene, protein, pdb
df = df.sort_values(["subsystem", "reaction_id", "gene_id", "uniprot_acc", "pdb_id"])

# Save full sequence to separate file if needed
df_sequences = df[["uniprot_acc", "full_sequence"]].drop_duplicates()
df_sequences.to_csv("protein_sequences.csv", index=False)

# Drop full sequence from main table (keep truncated version)
df = df.drop(columns=["full_sequence"])

# Save to CSV
df.to_csv("central_metabolism_master_table.csv", index=False)

print(f"Created master table with {len(df)} rows")
print(f"Columns ({len(df.columns)}): {list(df.columns)}")
print(f"\nBreakdown:")
print(f"  Unique reactions: {df['reaction_id'].nunique()}")
print(f"  Unique genes: {df['gene_id'].nunique()}")
print(f"  Unique proteins: {df['uniprot_acc'].nunique()}")
print(f"  Unique PDB structures: {df['pdb_id'].nunique()}")
print(f"  Rows with structures: {df['has_structure'].sum()}")

# Show sample
print("\nFirst few rows:")
print(df[["reaction_id", "gene_id", "uniprot_acc", "pdb_id", "subsystem"]].head(10))

Created master table with 1645 rows
Columns (18): ['reaction_id', 'reaction_name', 'reaction_equation', 'subsystem', 'metabolite_ids', 'num_metabolites', 'gene_id', 'gene_annotation', 'uniprot_acc', 'sequence_length', 'sequence', 'pdb_id', 'has_structure', 'total_genes_in_reaction', 'total_pdbs_for_protein', 'all_pdb_ids_for_protein', 'all_genes_for_protein', 'all_reactions_for_protein']

Breakdown:
  Unique reactions: 50
  Unique genes: 147
  Unique proteins: 756
  Unique PDB structures: 568
  Rows with structures: 983

First few rows:
   reaction_id          gene_id uniprot_acc pdb_id           subsystem
0       ACOAHi  ENSG00000172497      Q8WYK0   3B7K  Acetate production
1       ACOAHi  ENSG00000172497      Q8WYK0   4MOB  Acetate production
2       ACOAHi  ENSG00000172497      Q8WYK0   4MOC  Acetate production
60         ACS  ENSG00000131069      B4DEH9   None  Alcohol metabolism
61         ACS  ENSG00000131069      C9IYL0   None  Alcohol metabolism
62         ACS  ENSG00000131069